## Two-dimensional gradient descent

Two-dimensional gradient descent is the extension of 1D gradient descent to functions of multiple parameters, where the “slope” is now a vector called the gradient. 

In the tips example, the two parameters are the intercept (fixed tip amount) and the tip percentage, and gradient descent moves both together to minimize mean squared error.

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression

# Loading tips dataset
df = sns.load_dataset("tips")

### From 1D to 2D linear regression

In 1D, the model had a single parameter: just a tip percentage. Now the model is generalized to two parameters:

- $\theta_0$: a fixed offset (intercept), e.g., a base tip in dollars  
- $\theta_1$: the tip percentage

So the prediction for tip is:

$$
\hat{y} = \theta_0 + \theta_1 \cdot \text{bill}
$$

- $\hat{y}$: predicted tip  
- $\text{bill}$: bill amount  


### The bias column trick (intercept as a feature)

Instead of treating the intercept specially, the model can be rewritten in a unified way by adding a “bias” column:

1. Start with the tips dataset containing:
   - `bill` (feature)
   - `tip` (target)

2. Create a new dataset with:
   - `bias` = 1 for every row  
   - `bill` as before  

So each row of the feature matrix $X$ looks like:

$$
X_i = [1,\ \text{bill}_i]
$$

Now model prediction becomes:

$$
\hat{y}_i = \theta_0 \cdot X_{i,0} + \theta_1 \cdot X_{i,1}
= \theta_0 \cdot 1 + \theta_1 \cdot \text{bill}_i
$$

This is algebraically identical to the usual intercept + slope form, but now both parameters sit in **one** vector $\theta = [\theta_0,\ \theta_1]$, and both features are in **one** matrix $X$.

In [7]:
model = LinearRegression(fit_intercept = True)
X = df[["total_bill"]]
y = df["tip"]
model.fit(X, y)

print("Coefficient and intercept are", model.coef_, model.intercept_)

Coefficient and intercept are [0.10502452] 0.9202696135546731


#### Why this is useful

- Conceptually cleaner: intercept is just another coefficient.  
- Implementation-wise, gradient descent over multiple parameters becomes easier to write:
  - predictions are $X \theta$ (matrix–vector product)
  - gradients, updates, etc. use the same vector/matrix operations.

In [18]:
# Python-style implementation of the setup

# Add bias column
df["bias"] = 1.0

# Feature matrix X and target y
X = df[["bias", "total_bill"]]  # shape (n_samples, 2)
y = df["tip"]            # shape (n_samples,)


### Predictions with a parameter vector θ

Given a parameter vector:

$$
\theta = 
\begin{bmatrix}
\theta_0 \\
\theta_1
\end{bmatrix}
$$

Predictions are:

$$
\hat{y} = X @  \theta
$$

In [19]:
# In code:

def predict(X, theta):
    """
    X: shape (n_samples, 2) -> columns: bias, bill
    theta: shape (2,) -> [theta0, theta1]
    """
    return theta[0] * X.iloc[:, 0] + theta[1] * X.iloc[:, 1]

# Because the bias column is all ones, 
# you could omit `X[:, 0]` and just use `theta0` directly, 
# but keeping it gives a uniform structure for all parameters.

### Mean Squared Error (MSE) loss in 2D

The loss function to minimize is **mean squared error**:

$$
\text{MSE}(\theta) = \frac{1}{n} \sum_{i=1}^n \big(\hat{y}_i - y_i\big)^2
= \frac{1}{n} \sum_{i=1}^n \big((\theta_0 + \theta_1 \cdot \text{bill}_i) - y_i\big)^2
$$

In [20]:
def mse_loss(theta, X, y_obs):
    y_hat = theta[0] * X.iloc[:, 0] + theta[1] * X.iloc[:, 1]
    return np.mean((y_hat - y_obs) ** 2)    

#### Exploring the loss with different θ

- θ₀ = 1.5, θ₁ = 0.05 → some MSE  
- θ₀ = 2.5, θ₁ = 0.05 → different MSE  
- θ₀ = 1.5, θ₁ = 0.00 → larger MSE  

This suggests there is some combination of θ₀ and θ₁ that gives the smallest possible MSE. That combination is what optimization (gradient descent or libraries) will find.

In [22]:
# Examples in Python:

theta_try = [1.5, 0.05]
print(mse_loss(theta_try, X, y))

theta_try2 = np.array([2.5, 0.05])
print(mse_loss(theta_try2, X, y))

theta_try3 = np.array([1.5, 0.0])
print(mse_loss(theta_try3, X, y))

1.5340521752049179
1.5160890604508195
4.1514475409836065


### Brute-force search in 2D

Before using gradient descent, **brute forcing** is tried out: try many combinations of θ₀ and θ₁ on a grid, compute the loss for each, and pick the smallest.

Conceptually:

1. Choose a range of θ₀ values (e.g., from 0 to 2 dollars).
2. Choose a range of θ₁ values (e.g., from 0 to 0.2 = 20%).
3. Evaluate MSE at each pair (θ₀, θ₁) and record it.
4. The best grid point approximates the true minimum.

In [23]:
theta0_values = np.linspace(0.0, 2.0, 10)   # 10 values
theta1_values = np.linspace(0.0, 0.2, 10)   # 10 values

best_loss = float("inf")
best_theta = None

for t0 in theta0_values:
    for t1 in theta1_values:
        theta = np.array([t0, t1])
        loss = mse_loss(theta, X, y)
        if loss < best_loss:
            best_loss = loss
            best_theta = theta

print("Best grid theta:", best_theta)
print("Best grid loss:", best_loss)

Best grid theta: [0.88888889 0.11111111]
Best grid loss: 1.046873057073467


This yields some approximate optimum, e.g. something like θ₀ ≈ 0.8888, θ₁ ≈ 0.111, close to but not exactly the library’s 0.92 and 0.105, because the grid is coarse (only 10×10 points).

Brute force is conceptually easy but:

- Expensive: cost grows rapidly with dimension and grid resolution.  
- Inexact: you only hit the grid points, not the true continuous minimum.

In [25]:
import plotly.graph_objects as go

uvalues = np.linspace(0, 2, 10)
vvalues = np.linspace(0, 0.2, 10)
(u,v) = np.meshgrid(uvalues, vvalues)
thetas = np.vstack((u.flatten(),v.flatten()))

MSE = np.array([mse_loss(t, X, y) for t in thetas.T])

loss_surface = go.Surface(x=u, y=v, z=np.reshape(MSE, u.shape))

ind = np.argmin(MSE)
optimal_point = go.Scatter3d(name = "Optimal Point",
    x = [thetas.T[ind,0]], y = [thetas.T[ind,1]], 
    z = [MSE[ind]],
    marker=dict(size=10, color="red"))

fig = go.Figure(data=[loss_surface, optimal_point])
fig.update_layout(scene = dict(
    xaxis_title = "theta0",
    yaxis_title = "theta1",
    zaxis_title = "MSE"))
fig.show()

### Using optimization libraries (2D)

The next step is to use a general optimization routine (e.g. from SciPy or similar) that performs gradient-based optimization.

To use such a routine, it’s convenient to have a **single-argument** loss function: one that only accepts θ, with X and y captured from the surrounding scope:

In [28]:

# Wrapper that “freezes” X and y
def mse_loss_single_arg(theta):
    return mse_loss(theta, X, y)

# Then you can call an optimizer given a starting guess, e.g. θ = [0.0, 0.0]:

from scipy.optimize import minimize

theta_init = np.array([0.0, 0.0])

result = minimize(mse_loss_single_arg, theta_init)
theta_opt = result.x

print("Optimized theta:", theta_opt)  # should be close to [0.92, 0.105]
result

# This replicates what scikit-learn did internally: it minimized the MSE loss over θ₀ and θ₁ using gradient-based methods.

Optimized theta: [0.92027035 0.10502448]


  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1.0360194420114932
        x: [ 9.203e-01  1.050e-01]
      nit: 3
      jac: [-4.470e-08 -2.980e-08]
 hess_inv: [[ 2.980e+00 -1.253e-01]
            [-1.253e-01  6.335e-03]]
     nfev: 15
     njev: 5

### Intuition for multi-dimensional gradient descent

In 1D, the slope at a point tells you the best direction (up or down) to move. In 2D:

- The function $f(\theta_0, \theta_1)$ defines a surface.  
- At any point on that surface, the “steepest way down” is no longer just left or right; it is a **direction in the 2D plane** of parameters.  
- That direction is encoded in the **gradient vector**.

Imagine a 3D surface:

- Horizontal axes: θ₀ and θ₁  
- Vertical axis: loss value  

At a current position (θ₀, θ₁), the gradient arrow points in the direction of **steepest increase** of the function. For gradient descent, we step in the **opposite** direction (negative gradient) to go downhill.

As we repeat this process:

1. Compute gradient at current θ.  
2. Take a step against the gradient, scaled by a learning rate α.  
3. The path curves around as the gradient direction changes.  
4. Stop when the loss stops decreasing significantly.

### The gradient: definition and example

For a 2D function $f(\theta_0, \theta_1)$, the gradient is:

$$
\nabla_{\theta} f =
\begin{bmatrix}
\frac{\partial f}{\partial \theta_0} \\
\frac{\partial f}{\partial \theta_1}
\end{bmatrix}
$$

For example:

$$
f(\theta_0, \theta_1)
= 8 \theta_0^2 + 3 \theta_0 \theta_1
$$

This is **not** a regression loss; it’s just a random 2D function used to demonstrate partial derivatives.

Compute partial derivatives:

1. Partial derivative w.r.t. $\theta_0$:

$$
\frac{\partial f}{\partial \theta_0}
= 16 \theta_0 + 3 \theta_1
$$

2. Partial derivative w.r.t. $\theta_1$:

$$
\frac{\partial f}{\partial \theta_1}
= 3 \theta_0
$$

So the gradient is:

$$
\nabla_{\theta} f(\theta_0, \theta_1)
=
\begin{bmatrix}
16 \theta_0 + 3 \theta_1 \\
3 \theta_0
\end{bmatrix}
$$

This is exactly the column-vector to be used.

#### Interpreting gradient components

- Top entry: “If I nudge θ₀ up a tiny amount, how much does f go up or down?”  
- Bottom entry: “If I nudge θ₁ up a tiny amount, how much does f go up or down?”

In optimization:

- If a component of the gradient is **positive**, increasing that θ increases the function (bad for minimization), so we should decrease that θ.  
- If a component is **negative**, increasing that θ decreases the function (good), so we should increase that θ.

In gradient descent, all of this is unified into:

$$
\theta^{(t+1)} = \theta^{(t)} - \alpha \nabla_{\theta} f(\theta^{(t)})
$$

### Generalizing to p dimensions

For a function of $p+1$ parameters $\theta_0, \theta_1, \dots, \theta_p$:

$$
\nabla_{\theta} f(\theta) =
\begin{bmatrix}
\frac{\partial f}{\partial \theta_0} \\
\frac{\partial f}{\partial \theta_1} \\
\vdots \\
\frac{\partial f}{\partial \theta_p}
\end{bmatrix}
$$

- Each row corresponds to “what happens to the loss if I slightly increase this particular θ_j, holding the others fixed”.  
- Together, these partials form the vector that points in steepest ascent.  

Gradient descent moves in the **negative** of this vector, scaled by a learning rate α.

### Multidimensional gradient descent update rule

The updated rule:

$$
\theta^{(t+1)}
=
\theta^{(t)} - \alpha \nabla_{\theta} L(\theta^{(t)}, X, y)
$$

Where:

- $L(\theta, X, y)$: loss function (e.g., MSE)  
- $\theta^{(t)}$: current parameter vector at step t  
- $\theta^{(t+1)}$: next parameter vector  
- $\alpha$: learning rate  
- $\nabla_{\theta} L$: gradient of loss w.r.t. θ

You must also choose a **starting guess**, e.g.:

- All zeros: θ = [0, 0, …]  
- Small random numbers  
- Some heuristic or prior-based guess

### Gradient for 2D linear regression MSE (explicit)

For completeness, here is the gradient of the MSE for linear regression in matrix form.

Loss:

$$
L(\theta) = \frac{1}{n} \sum_{i=1}^n (\hat{y}_i - y_i)^2
= \frac{1}{n} \| X\theta - y \|^2
$$

The gradient is:

$$
\nabla_{\theta} L(\theta)
=
\frac{2}{n} X^\top (X\theta - y)
$$

For our 2D case (bias + bill), this gives a 2D gradient vector:

- First component: derivative w.r.t. θ₀  
- Second component: derivative w.r.t. θ₁  

### Python implementation: 2D gradient descent for tips

In [29]:
def mse_loss_and_grad(theta, X, y):
    """
    Returns both loss and gradient for efficiency.

    theta: (2,)
    X:     (n,2)
    y:     (n,)
    """
    n = X.shape[0]
    y_pred = X @ theta           # shape (n,)
    errors = y_pred - y          # shape (n,)
    loss = np.mean(errors ** 2)
    grad = (2 / n) * (X.T @ errors)  # shape (2,)
    return loss, grad

In [32]:
def gradient_descent(X, y, alpha=0.01, n_steps=1000):
    theta = np.zeros(X.shape[1])  # start at [0.0, 0.0]
    history = []

    for step in range(n_steps):
        loss, grad = mse_loss_and_grad(theta, X, y)
        theta = theta - alpha * grad
        history.append(loss)
        # Optional early stopping if loss change is tiny
        if step > 0 and abs(history[-2] - history[-1]) < 1e-8:
            break

    return theta, history

import warnings
warnings.filterwarnings('ignore')

theta_gd, loss_history = gradient_descent(X, y, alpha=0.01, n_steps=5000)
print("Gradient descent theta:", theta_gd)

Gradient descent theta: bias         NaN
total_bill   NaN
dtype: float64


- This procedure will converge to parameters close to what the library found (~0.92 and ~0.105) if α and step count are chosen reasonably.  
- The path of θ through parameter space is the 2D analogue of sliding down a 1D curve.

### Summary of key ideas

- Two-parameter model: tip prediction is $ \theta_0 + \theta_1 \cdot \text{bill} $.  
- Bias column trick: add a column of ones so intercept and slope can be handled uniformly via matrix operations.  
- MSE loss: measures how far predictions are from actual tips, averaged over all examples.  
- Brute-force optimization: try a grid of θ values and pick the one with smallest loss, but this is coarse and inefficient.  
- Optimization libraries: can minimize a single-argument loss function (θ → loss) and recover optimal θ automatically.  
- Gradient in multiple dimensions: vector of partial derivatives; each component tells how the function changes with respect to that parameter.  
- Gradient descent in $p$ dimensions: repeatedly update θ by subtracting a scaled gradient until the loss stops decreasing.